# LangGraph Meal Planning Agent with Bedrock Guardrails

## 🎯 Overview

This notebook demonstrates an **AI-powered meal planning system** that generates personalized recipe recommendations while ensuring food safety through **AWS Bedrock Guardrails**. The solution uses a multi-agent architecture built with LangGraph to process user preferences, generate recipes, and find cooking tutorials.

## 📋 Solution Description

The meal planning application analyzes natural language requests from users (e.g., "I want a vegan dish under 20 minutes") and generates three customized recipes complete with ingredients, instructions, and YouTube cooking tutorials. To ensure user safety, AWS Bedrock Guardrails actively monitor and block potentially harmful content related to:

- 🚫 Unsafe food handling practices
- 🔥 Dangerous cooking methods
- 🥜 Allergen negligence
- 🍖 Raw/undercooked high-risk foods
- 💊 False medical dietary claims
- ⚖️ Extreme dieting practices
- 🗑️ Spoiled food consumption

## 🤖 Multi-Agent Architecture

The system employs three specialized AI agents working in sequence:

### 1. **Meal Preference Analyzer Agent**
- **Input:** Natural language user request
- **Output:** Structured JSON with dietary preferences, restrictions, allergies, cuisine preferences, and cooking constraints
- **Model:** AWS Bedrock Nova Pro with guardrails
- **Purpose:** Converts free-form text into standardized format for recipe generation

### 2. **Recipe Generator Agent**
- **Input:** Structured JSON preferences
- **Output:** Three diverse recipes in markdown format (H2 headings for titles)
- **Tools:** DuckDuckGo search for recipe inspiration
- **Model:** AWS Bedrock Nova Pro with guardrails
- **Purpose:** Generates safe, customized recipes following all dietary requirements

### 3. **Video Finder Agent**
- **Input:** Extracted recipe titles (list)
- **Output:** YouTube cooking tutorial links for each recipe
- **Tools:** YouTube search API
- **Model:** AWS Bedrock Nova Pro with guardrails
- **Purpose:** Finds relevant cooking videos to help users prepare meals

## 🛡️ Guardrail Components

The guardrail system consists of four primary components that work together to ensure content safety:
![alt text](guardrails-1.png)
| Component | Description |
| --- | --- |
| Topic Policy | Filters input based on seven denied topics, preventing processing of prohibited content categories. |
| Content Filters | Analyzes content for hate speech, violence, or misconduct, blocking any content that violates these standards. |
| Word Filters | Scans for 17 specific food safety terms to ensure recipe-related content adheres to safety guidelines. |
| PII Protection | Anonymizes personally identifiable information to protect user privacy and comply with data protection regulations. |


## 🛡️ Multi-Agent Workflow with Guardrail Protection

The multi-agent system consists of three specialized agents:

![alt text](guardrails-agents.png)

| Agent | Role | Function |
| --- | --- | --- |
| Preference Analyzer | Agent 1 | Converts natural language requests into structured JSON format for further processing. |
| Recipe Generator | Agent 2 | Creates three recipes based on user preferences and performs relevant searches. |
| Video Finder | Agent 3 | Searches YouTube for tutorial videos related to the generated recipes. |






## 🔒 Guardrail Intervention Points

Guardrails protect users at **every stage** of the workflow:

| Stage | Protection | Example Blocked Content |
|-------|-----------|-------------------------|
| **User Input** | Scans incoming requests | "How to prepare raw chicken without cooking" |
| **Agent Output** | Monitors generated recipes | "Leave meat at room temperature overnight" |
| **Tool Calls** | Validates search queries | Blocks queries for dangerous techniques |

## 🌟 Key Features

✅ **Food Safety First** - Guardrails block 7 categories of dangerous food advice  
✅ **Personalized Recipes** - Respects dietary restrictions, allergies, and preferences  
✅ **Multi-Agent Orchestration** - LangGraph coordinates three specialized agents  
✅ **Visual Learning** - YouTube videos help users follow recipes step-by-step  
✅ **Real-time Protection** - Guardrails monitor every AI interaction  

---

Let's build a safe and intelligent meal planning assistant! 👨‍🍳

## ⚙️ Prerequisites

### 1. AWS Bedrock Guardrail Setup ⚠️ **REQUIRED**

You **must** run the `create_guardrail.ipynb` notebook first to create the AWS Bedrock Guardrail. This guardrail provides the safety layer that protects against dangerous food advice.

**The guardrail includes:**
- ✅ 7 denied topics (Unsafe Food Handling, Dangerous Cooking Methods, Allergen Negligence, Raw/Undercooked Foods, Medical Claims, Extreme Dieting, Spoiled Food)
- ✅ Content filters (Hate, Violence, Insults, Misconduct)
- ✅ 17 food safety word filters (poison, salmonella, moldy, expired, raw chicken, etc.)
- ✅ PII protection (Email, Phone, Name, Address anonymization)

**After creating the guardrail, you will receive:**
- **Guardrail ID**: e.g., `abc123xyz456`
- **Guardrail Version**: `1`

📝 **Note:** Update these values in **Step 2** of this notebook with your actual guardrail ID and version.

### 2. Install Required Dependencies

The following cell installs all required Python packages for this notebook.

**Key packages include:**
- `langchain>=1.0.0` - LangChain framework (v1.0+ required for `create_agent`)
- `langchain-community>=0.3.0` - Community tools (DuckDuckGo, YouTube search)
- `langchain-aws>=0.2.0` - AWS Bedrock integration
- `langgraph>=0.2.0` - Multi-agent orchestration
- `boto3>=1.34.0` - AWS SDK
- `ddgs>=6.0.0` - Web search functionality (formerly duckduckgo-search)
- `youtube-search>=2.1.0` - YouTube video search
- `ipython>=8.0.0` - Jupyter notebook support

In [ ]:
%pip install -r langgraph-guardrail-requirements.txt -qU

### 3. AWS Configuration

- **AWS Account**: Active account with Amazon Bedrock access
- **Bedrock Model Access**: Enabled access to `us.amazon.nova-pro-v1:0` model
- **AWS Region**: `us-east-1` (default in this notebook)

### 4. Verification Checklist

Before proceeding, verify:
- ☑️ `create_guardrail.ipynb` executed successfully
- ☑️ Guardrail ID and Version obtained from Step 4 of `create_guardrail.ipynb`
- ☑️ All packages installed from `langraph-guardrail-requirements.txt`
- ☑️ AWS Bedrock access configured

---

**Ready to start?** Let's import the libraries, configure the agents and prepare the meals

## Step 1: Import Required Libraries

In [ ]:
from langchain_aws import ChatBedrock
from langchain_community.tools import DuckDuckGoSearchRun, YouTubeSearchTool
from langchain.agents import create_agent
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from textwrap import dedent
from typing import TypedDict, Any
import re
from IPython.display import display, Markdown

## Step 2: Configure Guardrail and Initialize Model

Set up the guardrail ID and version (from the guardrail creation notebook), then initialize the Bedrock model with guardrail integration.

In [ ]:
# Guardrail Configuration
# ⚠️ IMPORTANT: Replace these with your actual guardrail ID and version from create_guardrail.ipynb
GUARDRAIL_ID = "<YOUR_GUARDRAIL_ID>"  # e.g., "abc123xyz456" - Get this from create_guardrail.ipynb output
GUARDRAIL_VERSION = "1"  # Version created in create_guardrail.ipynb

display(Markdown(f"✓ **Guardrail configured:** `{GUARDRAIL_ID}` (v{GUARDRAIL_VERSION})"))

In [ ]:
base_model = ChatBedrock(
    model="us.amazon.nova-pro-v1:0",
    temperature=0.7,
    region="us-east-1",
    guardrails={
        "guardrailIdentifier": GUARDRAIL_ID,
        "guardrailVersion": GUARDRAIL_VERSION,
        "trace": "enabled"  # Shows guardrail decisions for debugging
    }
)

display(Markdown("✓ **Bedrock model initialized with guardrail protection**"))

## Step 3: Define Tools

These tools enable our agents to access external information and enhance recipe generation:

- **🔍 Search Tool**: Provides recipe inspiration and cooking techniques via DuckDuckGo search. Used by the Recipe Generator Agent to find diverse recipe ideas.
- **📺 YouTube Search Tool**: Finds cooking tutorial videos for generated recipes. Used by the Video Finder Agent to help users learn preparation techniques.

In [ ]:
@tool
def search(query: str) -> str:
    """
    Search for information.
    """
    search_tool = DuckDuckGoSearchRun()
    results = search_tool.invoke(query, max_results=5)
    return f"Results for: {results}"

@tool
def youtube_search(query: str) -> str:
    """
    Search for YouTube cooking videos.
    """
    yt_tool = YouTubeSearchTool()
    results = yt_tool.run(query)
    return f"YouTube videos for '{query}':\n{results}"

## Step 4: Configure Meal Preference Analyzer Agent

This agent converts natural language user preferences into structured JSON format.

**How it works:**
- **Input:** Free-form text describing meal requirements (e.g., "I want a quick vegan dinner under 20 minutes")
- **Processing:** Extracts dietary preferences, restrictions, allergies, cuisine types, time constraints, and skill level
- **Output:** Clean JSON object with standardized fields for the Recipe Generator Agent
- **Default Values:** Infers reasonable defaults when information is missing (beginner skill level, $25 budget, 2 servings)

**Why this matters:** By standardizing user input into a consistent JSON format, we ensure the Recipe Generator Agent receives predictable, well-structured data regardless of how users phrase their requests. This separation of concerns makes the system more maintainable and allows each agent to focus on its specialized task.

In [ ]:
mealprefAgent_systemprompt=dedent("""You are the Meal Preference Analyzer Agent.
Your role is to take natural-language user input describing food preferences and convert it into a clean, 
structured JSON specification that another agent (the Recipe Generator Agent) can use to generate recipes.
Follow this strictly. If the user is ambiguous or has not provided an option, consider skill level: beginner, 
budget under $25 and servings of 2 and annotate it in additional_notes. For allergies, make a cautionary note 
regarding nuts. If no cuisine preferences were given, provide 3 different choices of popular cuisine. If no cooking 
time was given, assume 20-30 minutes. 

You MUST:
- Extract clear dietary rules, restrictions, allergies, and preferences.
- Identify cuisine interests, cooking time limits, skill level, and budget if mentioned.
- Infer reasonable defaults if the user does not specify them (state that these were inferred).
- Output ONLY valid JSON. Do not add any commentary or explanation.

Your JSON schema must match:

{
  "dietary_preferences": [],
  "avoid": [],
  "allergies": [],
  "preferred_cuisines": [],
  "cooking_time_minutes": null,
  "skill_level": null,
  "budget": null,
  "servings": null,
  "additional_notes": ""
}
""")

### Create Preference Agent

In [ ]:
preferenceAgent = create_agent(
    model=base_model,
    tools=[],  # No tools needed - this agent only parses text to JSON
    system_prompt=mealprefAgent_systemprompt
)

display(Markdown("✓ **Preference Analyzer Agent created**"))

## Step 5: Configure Recipe Generator Agent

This agent takes the structured JSON preferences and generates three diverse recipes in markdown format.

In [ ]:
recipeAgent_systemprompt = dedent("""You are the Recipe Generator Agent.

You receive structured meal preference JSON from another agent and your job is to generate THREE high-quality recipes that 
follow all the user's dietary requirements, restrictions, cuisine preferences, and cooking constraints.

IMPORTANT:
- Your output must be in **clean Markdown**, formatted for display using IPython's `display(Markdown(...))`.
- DO NOT output JSON.
- DO NOT wrap the output in code fences.
- DO NOT mention that you are an AI or explain the reasoning.
- Present the recipes clearly and visually using headings, bold text, bullet points, and numbered steps.

RECIPE RULES:
1. Provide EXACTLY three different recipes (unless impossible).
2. Each recipe must include:
   - A **recipe name** as an H2 Markdown heading (##).
   - A short **description**.
   - **Ingredients** in bullet points.
   - **Steps** in a numbered list.
   - **Estimated cooking time**.
   - **Notes** explaining how they satisfy the dietary preferences.
3. Respect:
   - allergies
   - avoid list
   - dietary preferences
   - preferred cuisines
   - maximum cooking time
   - skill level (if provided)
4. If any preference information is missing, make a reasonable assumption and note it under “Notes”.
5. Recipes must be diverse — no variations of the same dish.

Your entire output must be valid, readable Markdown with no JSON or code formatting.

            """)

### Create Recipe Agent

In [ ]:
recipeAgent = create_agent(
    model=base_model,
    tools=[search], 
    system_prompt=recipeAgent_systemprompt
)

display(Markdown("✓ **Recipe Generator Agent created**"))

## Step 6: Configure Video Finder Agent

This agent extracts recipe titles from the generated recipes and searches for related YouTube cooking videos.

In [ ]:
videoFinderAgent_systemprompt = dedent("""You are the Video Finder Agent.

You will receive a list of recipe titles. Your job is to find relevant YouTube cooking videos for EACH recipe.

IMPORTANT:
- You will receive recipe titles as a formatted list
- For EACH recipe title, use the youtube_search tool to find cooking tutorials
- The youtube_search tool takes a search query as input (e.g., "Chana Masala recipe tutorial")
- Format the output as clean Markdown with:
  - Recipe title as H3 heading (###)
  - List of YouTube video links and titles found
  - Brief note if no videos were found for that recipe
- Present the results in a clear, organized format

Your output must be valid Markdown that can be displayed to the user.
""")

### Create Video Finder Agent

In [ ]:
videoFinderAgent = create_agent(
    model=base_model,
    tools=[youtube_search],  # Use Langchain's YouTube Search tool
    system_prompt=videoFinderAgent_systemprompt
)

display(Markdown("✓ **Video Finder Agent created with Langchain YouTube Search**"))

## Step 7: Define Helper Functions

This section includes the response parser and title extractor functions.

**Why helper functions?**

The AWS Bedrock Nova model returns complex, structured responses that include multiple content blocks (text, tool calls, thinking tags). To work with these responses effectively, we need helper functions to:

1. **`parse_agent_response()`**: Extracts clean markdown text from Nova's structured response format
   - Handles different response types (strings, lists, dictionaries)
   - Removes internal `<thinking>` tags that aren't meant for user display
   - Normalizes whitespace for consistent formatting
   - Makes agent outputs ready for display or further processing

2. **`extract_recipe_titles()`**: Parses recipe titles from markdown content
   - Uses regex to find all H2 headings (`##`) which denote recipe titles
   - Cleans up markdown formatting (asterisks, whitespace)
   - Returns a clean list of titles for the Video Finder Agent
   - Enables automatic video search for each generated recipe

**Benefit:** These utilities keep our agent nodes clean and focused on orchestration logic, while handling the messy details of response parsing and data extraction in reusable functions.

In [ ]:
def parse_agent_response(raw_content) -> str:
    """
    Parse raw agent response to extract clean markdown text.

    This function handles AWS Nova's structured response format which can include
    multiple content blocks (text, tool_use, thinking tags). It extracts only the
    text content and removes thinking tags.

    """
    markdown_text = ""
    
    if isinstance(raw_content, str):
        markdown_text = raw_content
    elif isinstance(raw_content, list):
        for block in raw_content:
            if isinstance(block, dict) and block.get('type') == 'text':
                markdown_text += block.get('text', '')
    elif isinstance(raw_content, dict):
        markdown_text = raw_content.get('text', str(raw_content))

    markdown_text = re.sub(r'<thinking>.*?</thinking>', '', markdown_text, flags=re.DOTALL).strip()
    markdown_text = re.sub(r'\n{3,}', '\n\n', markdown_text)

    return markdown_text


def extract_recipe_titles(recipe_markdown: str) -> list:
    """
    Extract recipe titles from markdown content.
    
    Recipe titles are identified as H2 headings (##) in the markdown.

    """
    titles = re.findall(r'^##\s+(.+?)$', recipe_markdown, re.MULTILINE)
    
    cleaned_titles = []
    for title in titles:
        cleaned = title.strip().strip('*').strip()
        if cleaned:
            cleaned_titles.append(cleaned)
    
    return cleaned_titles

## Step 8: Build LangGraph Workflow

Define the state structure and create graph nodes for the three-agent workflow.

**Understanding the State Class:**

The `State` TypedDict defines the shared data structure that flows through all agents in the workflow:

- **`user_request`**: Original natural language input from the user
- **`preferences_json`**: Structured JSON output from the Preference Analyzer Agent
- **`generated_recipe`**: Markdown-formatted recipes from the Recipe Generator Agent
- **`recipe_titles`**: Extracted list of recipe names (H2 headings)
- **`video_links`**: YouTube tutorial links from the Video Finder Agent

Each agent node receives the current state, performs its task, updates relevant fields, and returns the modified state to the next agent.

**Agent Flow:**

```
User Input → [Preference Analyzer] → [Recipe Generator] → [Video Finder] → Final Output
             └─ Extracts JSON      └─ Creates 3 recipes   └─ Finds videos
```

**How it works:**
1. **Preference Analyzer** reads `user_request`, writes `preferences_json`
2. **Recipe Generator** reads `preferences_json`, writes `generated_recipe` and `recipe_titles`
3. **Video Finder** reads `recipe_titles`, writes `video_links`
4. Final state contains all outputs for display

This stateful approach ensures data flows seamlessly between agents while maintaining guardrail protection at every step.

In [ ]:
# Define State structure
class State(TypedDict, total=False):
    user_request: str
    preferences_json: str
    generated_recipe: str
    recipe_titles: list
    video_links: str
    
# -----------------------------------
# NODE 1 — Meal Preference Analyzer
# -----------------------------------
def run_preference_agent(context: State):
    """Parse user preferences into structured JSON."""
    user_input = context["user_request"]
    try:
        user_prompt = {"messages": [{"role": "user", "content": user_input}]}
        result = preferenceAgent.invoke(user_prompt, context={"user_role": "Human"})

        context["preferences_json"] = parse_agent_response(result['messages'][-1].content)
        return context
    except Exception as e:
        context["preferences_json"] = f"An unexpected error occurred: {e}"
        return context

# ----------------------------------
# NODE 2 — Meal Recipe Generator
# ----------------------------------
def run_recipe_agent(context: State):
    """Generate recipes from structured preferences."""
    pref_json = context["preferences_json"]
    try:
        user_prompt = {"messages": [{"role": "user", "content": f"Generate recipes for: {pref_json}"}]}
        result = recipeAgent.invoke(user_prompt, context={"user_role": "Human"})

        recipes = parse_agent_response(result['messages'][-1].content)
        context["generated_recipe"] = recipes

        context["recipe_titles"] = extract_recipe_titles(recipes)
        return context
    except Exception as e:
        context["generated_recipe"] = f"An unexpected error occurred: {e}"
        context["recipe_titles"] = []
        return context

# ----------------------------------
# NODE 3 — Video Finder
# ----------------------------------
def run_video_agent(context: State):
    """Find YouTube videos for the generated recipes."""
    recipe_titles = context.get("recipe_titles", [])
    
    if not recipe_titles:
        context["video_links"] = "⚠️ No recipe titles found to search for videos."
        return context

    try:
        titles_formatted = "\n".join([f"- {title}" for title in recipe_titles])
        prompt = f"Find YouTube cooking videos for these recipe titles:\n\n{titles_formatted}"
        
        user_prompt = {"messages": [{"role": "user", "content": prompt}]}
        result = videoFinderAgent.invoke(user_prompt, context={"user_role": "Human"})

        context["video_links"] = parse_agent_response(result['messages'][-1].content)
        return context
    except Exception as e:
        context["video_links"] = f"An unexpected error occurred while finding videos: {e}"
        return context

# ----------------------------------
# BUILD GRAPH
# ----------------------------------
graph = StateGraph(State)
graph.add_node("preferences", run_preference_agent)
graph.add_node("recipes", run_recipe_agent)
graph.add_node("videos", run_video_agent)

graph.set_entry_point("preferences")
graph.add_edge("preferences", "recipes")
graph.add_edge("recipes", "videos")
graph.add_edge("videos", END)

app = graph.compile()

display(Markdown("""
✓ **LangGraph workflow compiled successfully**

**Workflow:** `user_request` → `preferences` → `recipes` (extract titles) → `videos` → `output`
"""))

## Step 9: Run the Workflow

Execute the meal planning workflow with a sample user request.

In [ ]:
user_meal_input = "I want to make an middle eastern dish with rice, vegetables, meat and nuts. Don't worry about allergies "

final_state = app.invoke({"user_request": user_meal_input})

# Display the generated recipes
display(Markdown("# 🍽️ Generated Recipes"))
display(Markdown(final_state["generated_recipe"]))

# Display the YouTube video links
display(Markdown("---"))
display(Markdown("# 🎥 Cooking Video Tutorials"))
display(Markdown(final_state.get("video_links", "No video links available")))